In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/raw/44V-GEU314__Informe_Completo_2022-05-25_12-16-58_Seguimiento del sensor.csv', encoding='utf-8', sep=';', index_col=0)
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/44V-GEU314__Informe_Completo_2022-05-25_12-16-58_Seguimiento del sensor.csv'

## Data Cleaning

### replace default value '---' for NaN

In [ ]:
for column in df.select_dtypes('object').columns:
    df[column] = df[column].replace(r'----', np.NaN, regex=True)

### set up report dataframe

In [ ]:
#New report DataFrame
df_rep = pd.DataFrame()

#Date column
df_rep['datetime'] = pd.to_datetime(df['Fecha y Hora'], format='%Y-%m-%d %H:%M:%S')

#geo position columns
df_rep['latitude'] = df['Coordenadas'].str.split(',').str[0].astype('float32')
df_rep['longitude'] = df['Coordenadas'].str.split(',').str[1].astype('float32')
df_rep['lat_lon'] = df_rep['latitude'].round(3).astype('str') + ',' + df_rep['longitude'].round(3).astype('str')
df_rep['course'] = df['Curso*'].str.extract(r'(\d*.?\d{0,2})', expand=False).astype('float32')
df_rep['altitude'] = df['Altitud*'].str.extract(r'(\d*\.?\d{0,2})', expand=False)

#Ubication columns
df_rep['country'] = df['Localización'].str.split(',').str[1]
df_rep['city'] = df['Localización'].str.split(',').str[2]
df_rep['location'] = df['Localización'].str.split(',').str[0]
df_rep['address'] = df['Localización'].str.split(',', n=3).str[3]

#Battery columns
df_rep['battery_perc'] = df['Bateria Respaldo*'].str.extract(r'(\d*.?\d{0,2})', expand=False).astype('float32')/100
df_rep['battery_volt'] = df['Bateria Vehiculo*'].str.extract(r'(\d*.?\d{0,2})', expand=False).astype('float32')

#Sensor columns
df_rep['eng_eff'] = df['eng_eff_on_fuelconsmath*'].astype('float32')
df_rep['ignition'] = df['Ignicion*'].map({'Apagado': False, 'Encendido': True})#.astype('bool')
df_rep['millage'] = df['Kilometraje*'].str.extract(r'(\d*.?\d{0,2})', expand=False).astype('float32')
df_rep['velocity'] = df['Velocidad*'].str.extract(r'(\d*.?\d{0,2})', expand=False).astype('float32')

In [ ]:
df_rep = df_rep.set_index(pd.DatetimeIndex(df_rep['datetime']), drop=True)
df_rep.drop(columns=['datetime'], inplace=True)
df_rep.sample(10)

## Visualization and EDA

In [ ]:
from pandas_profiling import ProfileReport


In [ ]:
# import pandas_profiling as pp
# pp.version.__version__

In [ ]:
prof = ProfileReport(df_rep, minimal=True, title='Pandas Profiling Report')
# prof.to_file(output_file="./../reports/first_looks/report_1.html")

In [ ]:
prof

Here we can notice that there is a high number of 0's on velocity column (3792), let's see what can be happening, as a first approach we can review if this is related to time when the vehicle it's turned off

In [ ]:
df_vel_0 = df_rep[df_rep['velocity'] == 0]
df_vel_0.groupby('ignition').size()

As a first look there is noticiable difference between the amount of time that the vehicle was turned off and on whit velocity equals to zero, however here we didn't take into account that the sensors capture the velocity with less frequency while vehicle is turned off

In [ ]:
df_rep['time_range'] = df_rep.index.to_series().shift(-1) - df_rep.index.to_series()

In [ ]:
df_vel_0 = df_rep[df_rep['velocity'] == 0]
df_vel_0

In [ ]:
df_vel_0.groupby('ignition')['time_range'].sum()

Excelent, now we can notice that in fact this vehicle spended more than 19 days turned off vs 2.16 days turned on with velocity equals to zero. However this is a lot of time that can be being spend on traffic, so let's see what locations are involved on this situation to take actions

In [ ]:
df_traffic = df_rep[(df_rep['velocity'] <= 15) & (df_rep['ignition'] == True)]

# df_traffic.groupby(['country', 'city', 'location', 'lat_lon'])\
#     [['address','time_range']]\
#     .agg({'address': 'first', 'time_range': 'sum'})\
#     .sort_values(ascending=False, by='time_range')

df_traffic.groupby(['lat_lon'])\
    [['country', 'city', 'location', 'address','time_range']]\
    .agg({
        'country': 'first',
        'city': 'first',
        'location': 'first',
        'address': 'first',
        'time_range': 'sum'})\
    .sort_values(ascending=False, by='time_range')

In [ ]:
import folium  #needed for interactive map
from folium.plugins import HeatMap

In [ ]:
my_zip = zip(df_traffic['latitude'], df_traffic['longitude'], df_traffic['time_range'].dt.total_seconds())
list_of_my_zip = list(my_zip)

In [ ]:
hm_layer = HeatMap(list_of_my_zip,
                   # These are parameters that we tweak manually to adjust color
                   # See folium docs for more information
                   min_opacity=0.3,
                   radius=6,
                   blur=5,
                 )

In [ ]:
ny_coords = [4.65, -74.1] # lat, long
folium_map = folium.Map(location=ny_coords, zoom_start=12, tiles="OpenStreetMap")
folium_map.add_child(hm_layer)
folium_map

In [ ]:
df_vel_0[df_vel_0['ignition'] != True].groupby(['lat_lon'])\
    [['country', 'city', 'location', 'address','time_range']]\
    .agg({
        'country': 'first',
        'city': 'first',
        'location': 'first',
        'address': 'first',
        'time_range': 'sum'})\
    .sort_values(ascending=False, by='time_range')

In [ ]:
df_rep.sample()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sns.lineplot(data=df_rep.reset_index(), x='datetime', y='millage', ax=ax);

In [ ]:
df_vel_mean = df_rep[df_rep['ignition'] == True].groupby(['location'])[['velocity']].mean()
df_vel_mean.sort_values(by='velocity', ascending=False, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sns.barplot(data=df_vel_mean.reset_index(), x='location', y='velocity', ax=ax)
ax.tick_params(labelrotation=45)
plt.show();

In [ ]:
df_traffic_days = df_traffic.copy()
df_traffic_days['day'] = df_traffic_days.index.weekday

In [ ]:
import calendar

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sns.countplot(data=df_traffic_days.reset_index(), x='day', ax=ax)
ax.set_xticklabels(calendar.day_name)
plt.show();